# Power-system State-Estimation using Gurobi 

This notebook will take you through the steps required to perform custom Weighted Least Squares State-Estimation for a few pandapower networks. 

Methodology: 



In [14]:
import gurobipy as gp 
import logging 
import os 
import pandapower as pp 
import numpy as np 
import pandas as pd 
from utils import get_branch_parameters, get_edge_index_from_ppnet, get_neighbor_dict
from gurobipy import nlfunc
from gurobipy import GRB

# configure logging
log_dir = "logs"
log_file = "script_log.txt"

# ensure log directory exists
os.makedirs(log_dir, exist_ok=True)

logging.basicConfig(
    filename=os.path.join(log_dir, log_file),
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)

env = gp.Env(params={"LogToConsole": 0})

## Consider a 4-Bus Network: 

<img src="4_bus_network/4_bus_network.001.jpeg" width="500">

In [15]:
net = pp.create_empty_network()

bus1 = pp.create_bus(net, vn_kv=20.0, name="Bus 1")
bus2 = pp.create_bus(net, vn_kv=20.0, name="Bus 2")
bus3 = pp.create_bus(net, vn_kv=0.4, name="Bus 3")  # connected via transformer
bus4 = pp.create_bus(net, vn_kv=20.0, name="Bus 4")

nodes = net.bus.index.values

# standard 20 kV line type 
std_line_type = "NAYY 4x50 SE"

# Create standard lines
pp.create_line(net, from_bus=bus1, to_bus=bus2, length_km=1.0,
               std_type=std_line_type, name="Line 1-2")

pp.create_line(net, from_bus=bus1, to_bus=bus4, length_km=1.0,
               std_type=std_line_type, name="Line 1-4")

pp.create_line(net, from_bus=bus2, to_bus=bus4, length_km=1.0,
               std_type=std_line_type, name="Line 2-4")

pp.create_transformer(net, bus2, bus3, std_type="0.25 MVA 20/0.4 kV", name="Trafo 2-3")
net.trafo.shift_degree = 0.0 # bug in pandapwer 

pp.create_ext_grid(net, bus=bus2, vm_pu=1.02, name="Grid Connection")

pp.create_load(net, bus3, p_mw=0.2, q_mvar=0.05, name="Load Bus 3")
pp.create_load(net, bus4, p_mw=0.3, q_mvar=0.1, name="Load Bus 4")

pp.runpp(net)

edge_index = get_edge_index_from_ppnet(net=net)

### Initialize the variables

In [16]:
# how many buses are there? 
num_buses = len(net.bus.index)

# how many state-variables 
n = 2*num_buses - 1 # \theta_1 = 0 for reference 

# consider fully-observable network, so number of measurements >= number of states 
# using two measurements of active power and voltage at each bus 
m = 2*num_buses

# initialize model 
gp_model = gp.Model("WLS_State_Estimation")

# measurement vector as 5% tolerance to power flow results 
V_meas = np.random.normal(net.res_bus.vm_pu.values, 0.05/3)
P_meas = np.random.normal(net.res_bus.p_mw.values, 0.05/3)

# Gurobi variables for estimated voltages and angles
# subject to voltage violations and angle violations 
X_vm_pu = {i: gp_model.addVar(name=f"|V|_bus{i}") for i in range(num_buses)}
X_va_rad = {i: gp_model.addVar(name=f"A_bus{i}") for i in range(num_buses)}

# slack bus angle 
X_va_rad[0].LB, X_va_rad[0].UB = 0.0, 0.0 
gp_model.update()


In [18]:
# get line and transformer parameters
# param_dict = {(i, j): [G_ij, B_ij, g_s_ij, b_s_ij, g_sh_ij, b_sh_ij]}
# TODO: add tap-changing transformers and mutual coupling between lines
param_dict = get_branch_parameters(net=net) 
param_dict

{(0, 1): [-612.8103127766655,
  79.22625539012964,
  612.8103127766655,
  -79.22625539012964,
  0.0,
  -0.02638937829015426],
 (0, 3): [-612.8103127766655,
  79.22625539012964,
  612.8103127766655,
  -79.22625539012964,
  0.0,
  -0.02638937829015426],
 (1, 3): [-612.8103127766655,
  79.22625539012964,
  612.8103127766655,
  -79.22625539012964,
  0.0,
  -0.02638937829015426],
 (1, 2): [-0.9998000023044077,
  4.044887023851327,
  0.9998000023044077,
  -4.044887023851327,
  0.0007999907823690311,
  -3.7276819853104826e-08],
 (1, 0): [-612.8103127766655,
  79.22625539012964,
  612.8103127766655,
  -79.22625539012964,
  0.0,
  -0.02638937829015426],
 (3, 0): [-612.8103127766655,
  79.22625539012964,
  612.8103127766655,
  -79.22625539012964,
  0.0,
  -0.02638937829015426],
 (3, 1): [-612.8103127766655,
  79.22625539012964,
  612.8103127766655,
  -79.22625539012964,
  0.0,
  -0.02638937829015426],
 (2, 1): [-0.9998000023044077,
  4.044887023851327,
  0.9998000023044077,
  -4.044887023851327,

In [19]:
# get neighbor_dictionary 
# neighbor_dict = {src: [neighbors]}
neighbor_dict = get_neighbor_dict(edge_index)
neighbor_dict

{0: [1, 3], 1: [0, 2, 3], 3: [0, 1], 2: [1]}

In [ ]:
# create expressions for power injection and voltage 

# gp_model.addConstr(
#     (X_vm_pu[i]*X_va_deg[j]()) for j in neighbor_dict[i] for i in nodes, 
#     name=f"P_meas_{i}"

# )

# Gurobi variables for estimated voltages and angles
# subject to voltage violations and angle violations 
X_vm_pu = {i: gp_model.addVar(name=f"|V|_bus{i}") for i in range(num_buses)}
X_va_rad = {i: gp_model.addVar(name=f"A_bus{i}") for i in range(num_buses)}

# slack bus angle 
X_va_rad[0].LB, X_va_rad[0].UB = 0.0, 0.0 
gp_model.update()

P_meas_expr = {int(i): [] for i in nodes}
V_meas_expr = {int(i): [] for i in nodes}

for i in nodes: 
    i = int(i)
    P_meas_i = 0.0
    for j in neighbor_dict[i]:
        j = int(j)
        P_meas_i += X_vm_pu[i] * X_vm_pu[j] * (param_dict[(i,j)][0]*nlfunc.cos(X_va_rad[i] - X_va_rad[j]) + 
                                               param_dict[(i,j)][1]*nlfunc.sin(X_va_rad[i] - X_va_rad[j]))
    # capture the expressions 
    P_meas_expr[i] = P_meas[i] + P_meas_i 
    V_meas_expr[i] = X_vm_pu[i] - V_meas[i]
    # gp_model.addConstr(0 == P_meas_i - P_meas[i], name=f"P_meas[{i}]")
    # gp_model.addConstr(0 == V_meas[i] - X_vm_pu[i], name=f"V_meas[{i}]")
    # break

P_meas_expr, V_meas_expr

({0: <gurobi.NLExpr: -0.006028269834233913 + 0.0 + |V|_bus0 * |V|_bus1 * (-612.8103127766655 * cos(A_bus0 + -1.0 * A_bus1) + 79.22625539012964 * sin(A_bus0 + -1.0 * A_bus1)) + |V|_bus0 * |V|_bus3 * (-612.8103127766655 * cos(A_bus0 + -1.0 * A_bus3) + 79.22625539012964 * sin(A_bus0 + -1.0 * A_bus3))>,
  1: <gurobi.NLExpr: -0.49769935907160945 + 0.0 + |V|_bus1 * |V|_bus0 * (-612.8103127766655 * cos(A_bus1 + -1.0 * A_bus0) + 79.22625539012964 * sin(A_bus1 + -1.0 * A_bus0)) + |V|_bus1 * |V|_bus2 * (-0.9998000023044077 * cos(A_bus1 + -1.0 * A_bus2) + 4.044887023851327 * sin(A_bus1 + -1.0 * A_bus2)) + |V|_bus1 * |V|_bus3 * (-612.8103127766655 * cos(A_bus1 + -1.0 * A_bus3) + 79.22625539012964 * sin(A_bus1 + -1.0 * A_bus3))>,
  2: <gurobi.NLExpr: 0.17759383670447107 + 0.0 + |V|_bus2 * |V|_bus1 * (-0.9998000023044077 * cos(A_bus2 + -1.0 * A_bus1) + 4.044887023851327 * sin(A_bus2 + -1.0 * A_bus1))>,
  3: <gurobi.NLExpr: 0.3325629357843368 + 0.0 + |V|_bus3 * |V|_bus0 * (-612.8103127766655 * cos(A_

In [21]:
objective = sum(P_meas_expr.values()) + sum(V_meas_expr.values())

gp_model.setObjective(objective, GRB.MINIMIZE)

GurobiError: Objective must be linear or quadratic

In [23]:
gp_model.setParam("NonConvex", 2)  # Allows non-convex MINLP problems

# Use a variable to represent the nonlinear objective value
obj_var = gp_model.addVar(name="objective", lb=-GRB.INFINITY)


# Set obj_var equal to the full nonlinear expression
gp_model.addGenConstrNL(obj_var, sum(P_meas_expr.values()) + sum(V_meas_expr.values()), "=", name="obj_def")

# Then minimize that variable
gp_model.setObjective(obj_var, GRB.MINIMIZE)


Set parameter NonConvex to value 2


TypeError: addGenConstrNL() got multiple values for keyword argument 'name'

In [ ]:
P_meas_expr = {}
V_meas_expr = {}

for i in nodes:
    # Build nonlinear expression symbolically
    expr = 0.0
    for j in neighbor_dict[i]:
        # only if (i,j) in 
        delta_theta = X_va_rad[i] - X_va_rad[j]
        print(param_dict[(int(i), int(j))][0], param_dict[(int(i), int(j))][1])
        Gij, Bij = param_dict[(i, j)][0], param_dict[(i, j)][1]
        expr += X_vm_pu[i] * X_vm_pu[j] * (Gij * nlfunc.cos(delta_theta) + Bij * nlfunc.sin(delta_theta))
    
    # Store the symbolic expressions
    P_meas_expr[i] = expr
    V_meas_expr[i] = X_vm_pu[i] - V_meas[i]


-612.8103127766655 79.22625539012964
-612.8103127766655 79.22625539012964


KeyError: (1, 0)

In [23]:
neighbor_dict, param_dict

({0: [1, 3], 1: [0, 2, 3], 3: [0, 1], 2: [1]},
 {(0, 1): [-612.8103127766655,
   79.22625539012964,
   612.8103127766655,
   -79.22625539012964,
   0.0,
   -0.02638937829015426],
  (0, 3): [-612.8103127766655,
   79.22625539012964,
   612.8103127766655,
   -79.22625539012964,
   0.0,
   -0.02638937829015426],
  (1, 3): [-612.8103127766655,
   79.22625539012964,
   612.8103127766655,
   -79.22625539012964,
   0.0,
   -0.02638937829015426],
  (1, 2): [-0.9998000023044077,
   4.044887023851327,
   0.9998000023044077,
   -4.044887023851327,
   0.0007999907823690311,
   -3.7276819853104826e-08]})